In [1]:
print("Hey Roopa ! Great to see you here")


Hey Roopa ! Great to see you here


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
PINECONE_API_KEY = "9a068d8b-d1c4-458a-97c4-969589ebe663"
PINECONE_API_ENV = "Serverless"